<a href="https://colab.research.google.com/github/dhwlxor/My-Ropo/blob/main/%EA%B2%AC%EC%A2%85%EC%9D%B8%EC%8B%9D_%ED%95%99%EC%8A%B5_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Rescaling, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import image_dataset_from_directory
import pathlib
import os
import tarfile

# STEP 1. 압축 해제
uploaded_path = '/content/images.tar'
with tarfile.open(uploaded_path) as tar:
    tar.extractall(path='./datasets')

data_path = pathlib.Path('datasets/Images')  # 대소문자 주의

# STEP 2. 데이터셋 생성
train_ds = image_dataset_from_directory(data_path,
                                        validation_split=0.2,
                                        subset='training',
                                        seed=123,
                                        image_size=(224, 224),
                                        batch_size=8)

test_ds = image_dataset_from_directory(data_path,
                                       validation_split=0.2,
                                       subset='validation',
                                       seed=123,
                                       image_size=(224, 224),
                                       batch_size=8)

# STEP 3. 모델 구성
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # 전이학습 성능 안정화를 위해 freezing 가능

cnn = Sequential([
    Rescaling(1.0 / 255.0),
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(120, activation='softmax')
])

cnn.compile(loss='sparse_categorical_crossentropy',
            optimizer=Adam(learning_rate=1e-5),
            metrics=['accuracy'])

hist = cnn.fit(train_ds, epochs=200, validation_data=test_ds, verbose=2)

# STEP 4. 저장
cnn.save('cnn_for_stanford_dogs.h5')

# 클래스 이름 저장
import pickle
with open('dog_species_names.pkl', 'wb') as f:
    pickle.dump(train_ds.class_names, f)

# STEP 5. 그래프 출력
import matplotlib.pyplot as plt

plt.plot(hist.history['accuracy'], label='Train Acc')
plt.plot(hist.history['val_accuracy'], label='Val Acc')
plt.legend(); plt.grid(); plt.title("Accuracy"); plt.show()

plt.plot(hist.history['loss'], label='Train Loss')
plt.plot(hist.history['val_loss'], label='Val Loss')
plt.legend(); plt.grid(); plt.title("Loss"); plt.show()

from google.colab import files

# 모델 다운로드
files.download('cnn_for_stanford_dogs.h5')

# 클래스 이름 다운로드
files.download('dog_species_names.pkl')


FileNotFoundError: [Errno 2] No such file or directory: '/content/images.tar'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
